## Abrir dataset Coswara

In [1]:
import pandas as pd

# Cargar el CSV con la codificación 'ISO-8859-1'
csv_path = 'CoronaHack-Respiratory-Sound-Dataset/Corona-Hack-Respiratory-Sound-Metadata.csv'  # Cambia a la ruta correcta de tu archivo CSV
data = pd.read_csv(csv_path, encoding='ISO-8859-1')

# Mostrar las primeras filas para verificar la carga correcta
print(data.head())


                        USER_ID COUNTRY  AGE COVID_STATUS ENGLISH_PROFICIENCY  \
0  vK2bLRNzllXNeyOMudnNSL5cfpG2   India   24      healthy                   Y   
1  bjA2KpSxneNskrLBeqi4bqoTDQl2   India   72      healthy                   Y   
2  FSzobvJqOXf0rI6X05cHqOiU9Mu2   India   54      healthy                   Y   
3  EqDWckxbsETyHUeBLQ8jLtxlhir2   India   31      healthy                   Y   
4  FGRDO4IBbAejR0WHD5YbkXTCasg2   India   26      healthy                   Y   

  GENDER COUNTY_RO_STATE CITY_LOCALITY  Diabetes  Asthma  ...     DATES  \
0      M       Karnataka     Bangalore         0       0  ...  20200413   
1      M     Maharashtra         Thane         0       0  ...  20200413   
2      M     Maharashtra    Thane West         0       0  ...  20200413   
3      M       Karnataka     Bangalore         0       0  ...  20200413   
4      M         Haryana       gurgaon         0       0  ...  20200413   

                                      breathing-deep  \
0  /da

## Filtrar por claves necesarias

In [6]:
columnas_necesarias = [
    'USER_ID', 'AGE', 'GENDER', 'COVID_STATUS', 'counting-fast'
]

# Eliminar todas las demás columnas excepto las que están en 'columnas_necesarias'
data_filtrada = data[columnas_necesarias]

# Mostrar las primeras filas para verificar
print(data_filtrada.head())


                        USER_ID  AGE GENDER COVID_STATUS  \
0  vK2bLRNzllXNeyOMudnNSL5cfpG2   24      M      healthy   
1  bjA2KpSxneNskrLBeqi4bqoTDQl2   72      M      healthy   
2  FSzobvJqOXf0rI6X05cHqOiU9Mu2   54      M      healthy   
3  EqDWckxbsETyHUeBLQ8jLtxlhir2   31      M      healthy   
4  FGRDO4IBbAejR0WHD5YbkXTCasg2   26      M      healthy   

                                       counting-fast  
0  /data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...  
1  /data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...  
2  /data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...  
3  /data/train/20200413/EqDWckxbsETyHUeBLQ8jLtxlh...  
4  /data/train/20200413/FGRDO4IBbAejR0WHD5YbkXTCa...  


## Cambiar etiquetas a claves numericas

In [7]:
# Crear un diccionario de mapeo
etiquetas = {'healthy': 0,
            'recovered_full': 0, 
            'positive_asymp': 1, 
            'positive_mild': 1, 
            'positive_moderate': 1,
            'resp_illness_not_identified':1,
            'no_resp_illness_exposed':1}

# Mapear las etiquetas
data_filtrada['COVID_STATUS'] = data_filtrada['COVID_STATUS'].map(etiquetas).fillna(0).astype(int)

C:\Users\Vicen\AppData\Local\Temp\ipykernel_18720\1067565498.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtrada['COVID_STATUS'] = data_filtrada['COVID_STATUS'].map(etiquetas).fillna(0).astype(int)


## Guardar nuevo dataframe

In [8]:
# Guardar el DataFrame filtrado en un nuevo archivo CSV

output_csv_path = 'nuevo_dataset.csv'  # Cambia a la ruta y nombre de tu elección
data_filtrada.to_csv(output_csv_path, index=False)


## Pasar los audios a espectrogramas

In [11]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Cargar el dataset filtrado (ajusta la ruta según sea necesario)
csv_path = 'nuevo_dataset.csv'
data_filtrada = pd.read_csv(csv_path)

# Directorios de salida para guardar los espectrogramas
output_dir_positive = 'espectrogramas/positivo'
output_dir_negative = 'espectrogramas/negativo'
os.makedirs(output_dir_positive, exist_ok=True)
os.makedirs(output_dir_negative, exist_ok=True)

# Función para dividir el audio en segmentos
def dividir_audio(y, sr, dur_segmento=2.5):
    segmentos = []
    dur_total = librosa.get_duration(y=y, sr=sr)
    for i in range(0, int(dur_total // dur_segmento)):
        start = int(i * dur_segmento * sr)
        end = int((i + 1) * dur_segmento * sr)
        segmentos.append(y[start:end])
    return segmentos

# Función para generar espectrograma segmentado
def generar_espectrograma_segmentado(audio_path, output_path):
    if not os.path.isfile(audio_path):
        print(f"Archivo no encontrado: {audio_path}")
        return

    try:
        y, sr = librosa.load(audio_path, sr=None)  # Mantener la frecuencia de muestreo original
        segmentos = dividir_audio(y, sr)  # Dividir audio en segmentos

        # Crear figura para espectrograma segmentado
        plt.figure(figsize=(8, 4))
        is_valid_spectrogram = False  # Bandera para determinar si el espectrograma es válido
        
        for i, segmento in enumerate(segmentos):
            # Verificar que el segmento no esté vacío ni sea silencioso
            if np.max(np.abs(segmento)) < 1e-3:
                print(f"Segmento {i} silencioso, omitiendo...")
                continue
            
            # Normalizar el segmento
            segmento = segmento / np.max(np.abs(segmento)) if np.max(np.abs(segmento)) > 0 else segmento
            
            # Generar espectrograma para cada segmento
            S = librosa.feature.melspectrogram(y=segmento, sr=sr, n_mels=128, fmax=8000)
            S_dB = librosa.power_to_db(S, ref=np.max)
            S_dB = np.clip(S_dB, -80, 0)  # Limitar valores entre -80 dB y 0 dB
            
            # Validar si el espectrograma tiene suficiente variación
            if np.std(S_dB) < 1:  # Baja variación indica espectrograma inválido
                print(f"Segmento {i} tiene baja variación en el espectrograma, omitiendo...")
                continue

            is_valid_spectrogram = True  # Al menos un segmento es válido

            # Mostrar espectrograma en su posición
            plt.subplot(1, len(segmentos), i + 1)
            librosa.display.specshow(S_dB, sr=sr, x_axis=None, y_axis=None, cmap='jet')
            plt.axis('off')

        if not is_valid_spectrogram:
            print(f"Espectrograma no válido para archivo: {audio_path}")
            plt.close()
            return  # No guardar si ningún segmento es válido

        plt.tight_layout(pad=0)
        plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close()
    except Exception as e:
        print(f"Error procesando {audio_path}: {e}")

# Recorrer cada fila del DataFrame y procesar cada columna de audio
cols_to_process = ['counting-fast']

for index, row in data_filtrada.iterrows():
    user_id = row['USER_ID']
    covid_status = row['COVID_STATUS']
    
    # Verificar el valor de covid_status
    print(f"COVID_STATUS para usuario {user_id}: {covid_status}")
    
    for col in cols_to_process:
        audio_path = row[col]
        if pd.notna(audio_path):
            audio_path = os.path.join("CoronaHack-Respiratory-Sound-Dataset" + audio_path)
            audio_path = os.path.abspath(audio_path)
            
            # Seleccionar el directorio según el estado de COVID
            if str(covid_status) == '1':  # Asumir que '1' significa COVID positivo
                output_dir = output_dir_positive
            else:
                output_dir = output_dir_negative
           
            output_path = os.path.join(output_dir, f"{user_id}_{covid_status}_{col}.png")
            if os.path.exists(output_path):
                print(f"Espectrograma ya existe: {output_path}")
                continue
            
            print(f"Procesando archivo: {audio_path}")
            generar_espectrograma_segmentado(audio_path, output_path)
            print(f"Espectrograma guardado en: {output_path}")


COVID_STATUS para usuario vK2bLRNzllXNeyOMudnNSL5cfpG2: 0
Espectrograma ya existe: espectrogramas/negativo\vK2bLRNzllXNeyOMudnNSL5cfpG2_0_counting-fast.png
COVID_STATUS para usuario bjA2KpSxneNskrLBeqi4bqoTDQl2: 0
Espectrograma ya existe: espectrogramas/negativo\bjA2KpSxneNskrLBeqi4bqoTDQl2_0_counting-fast.png
COVID_STATUS para usuario FSzobvJqOXf0rI6X05cHqOiU9Mu2: 0
Espectrograma ya existe: espectrogramas/negativo\FSzobvJqOXf0rI6X05cHqOiU9Mu2_0_counting-fast.png
COVID_STATUS para usuario EqDWckxbsETyHUeBLQ8jLtxlhir2: 0
Espectrograma ya existe: espectrogramas/negativo\EqDWckxbsETyHUeBLQ8jLtxlhir2_0_counting-fast.png
COVID_STATUS para usuario FGRDO4IBbAejR0WHD5YbkXTCasg2: 0
Espectrograma ya existe: espectrogramas/negativo\FGRDO4IBbAejR0WHD5YbkXTCasg2_0_counting-fast.png
COVID_STATUS para usuario htQzROl26OWQpIYFDzv11F79PLR2: 0
Procesando archivo: c:\Users\Vicen\OneDrive\Escritorio\Covid\DeteccionCovidRespiracion\CoronaHack-Respiratory-Sound-Dataset\data\train\20200413\htQzROl26OWQpIYFDz